# Setting the ROOT environment and read files

In this first example we will go over some useful methods to open files and have a quick look to the data stores inside.


The file corresponds with a very small fraction of data from the ```/DoubleMuon/``` dataset of 2022 data i.e. collisions collected by muon triggers. It contains data in NanoAOD format, which is the type of data that results from the whole processing chain in CMS. 

To know about the NanoAOD format and the kind of information that is stored inside, you can check the CMS [WorkBookNanoAOD](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookNanoAOD).

## Setup the environment

We will start by setting up the environment. These tutorials will be done in PyROOT, so will will have to import the Python ROOT package.

Note: ROOT programming with c++ is not covered here.

In [ ]:
import ROOT

## Reading a file

Data in ROOT is included in ```.root``` files with ROOT formats. Every time that we want to read a file, we first have to create a TFile object from which we can open it. We can then explore the contents of the TFile by calling ```.ls()```:

In [ ]:
# Read the file
file = ROOT.TFile.Open("/eos/user/f/fernance/standard-TutoriasFiles/43718dea-5cd4-48a7-b73b-df168edf1fac.root")

# Show what it is inside
file.ls()

As we can see there are different objects stored in the file. For now we will focus on one of the TTree objects, called "Events". As the name indicates, this TTree contains the events stored in this file.

## TTrees

We can get this TTree with the method ```Get()``` indicating its name ```"Events"```:

In [ ]:
# Get the TTree
ttree = file.Get('Events')

### Quick exploration of the TTree content

We can explore the content of the TTree and the branch structure by calling ```Print()``` on the TTree object. This will let us know the type of data stored in them, the size occupied by each branch and other useful information:

*Note: For now this example only covers plan data formats. That is, the data that we are accessing are just arrays of integers (Int_t), floats (Float_t) or booleans (Bool_t)*

In [ ]:
# Print TTree content
ttree.Print()

Since there are too many branches the output is very long. We can select the ones that we want to print by typing fractions of their name:

In [ ]:
# Print TTree content only for Muon branches
ttree.Print("Muon_*")

We can also check the values of the different branches with the method ```Scan()``` that show the data in a columnar way. However, since there are too many branches the information will only be plotted for the first ones:

*Note: We are only showing the information of the first 10 events of the TTree*

In [ ]:
ttree.Scan("", "", "", 10)

In the following examples we will explore some muon variables. We will have a look at some of them to get familiar with the data structure. This can be done by specifying the branches we want to read delimited by ```:``` e.g.

In [ ]:
ttree.Scan("Muon_pt:Muon_eta:Muon_phi", "", "", 10)

In the example above, each ```Row``` represents one event. Within each row/event, the printed objects are indicated as ```Instance```. From this printout we can know that the first event (Row = 0) has two muons (Instances 0 and 1); the second event (Row = 1) has three muons (Instances 0, 1 and 2)... etc as well as their values of transverse momentum, pseudorapidity and azimuthal angle.

### Setting aliases on an existing TTree

In many occasions we will want to get more complicated physical observables that are not in the TTree, but that can be derived from information that is already stored inside. If the observable is not much complex we can define an alias that will compute the observable that will be available in the TTree.

For example, we have the values of pt, eta and phi for every muon. From these values we could compute the invariant mass for the first two muons of the event and define an alias:

In [ ]:
ttree.SetAlias('Dimuon_mass', 'sqrt(2*Muon_pt[0]*Muon_pt[1]*(cosh(Muon_eta[0] - Muon_eta[1]) - cos(Muon_phi[0] - Muon_phi[1])))')

And then show it in an scan in the same way as before:

In [ ]:
ttree.Scan("Muon_pt:Muon_eta:Muon_phi:Dimuon_mass", "", "", 10)

See how know the last collumn is shown the invariant mass computed with the first two muons of the event. This is a "per event defined observable" so we will have it once per row and duplicated for each instance. In cases where we don't have at least two muons to compute it from, it will be left empty.

### Loop over the TTree events

For more complicated actions on the TTree, we will need to loop over the events, this can be done in many ways, here we have two equivalent examples:

In [ ]:
# Numer of entries of the TTree
nEvents = ttree.GetEntries()
print("TTree 'Events' has %i events"%nEvents)

In [ ]:
# Loop over TTree by accessing a given entry
for e in range(0, 10):
    ttree.GetEntry(e)
    nMuon = ttree.nMuon
    print("Event %i has %i muons"%(e, nMuon))

In [ ]:
# Loop over the events of a TTree
for e,event in enumerate(ttree):
    if e>9:
        break
    nMuon = event.nMuon
    print("Event %i has %i muons"%(e, nMuon))
    e+=1

## Quick draw in TTree

If we just want to explore how variables are distributed, we can quickly draw them using the TTree (including aliases). When doing this in interactive root in the terminal the plot will just appear (not covered here) but when using PyROOT we have to create the canvas ourselves. This is a TCanvas ROOT object that we can manipulate:

In [ ]:
# Canvas creation:
canvas = ROOT.TCanvas("canvas", "My first canvas", 600, 500) # 600 and 500 and width and height of the canvas
canvas.cd() # To indicate we are going to draw here

# draw:
ttree.Draw("Muon_pt")

# and save into a png which can be do in two different ways:
canvas.SaveAs("MyFirstPlot_SaveAs.png")
canvas.Print("MyFirstPlot_Print.png")

ONLY to see them in the notebooks we will use the Python IPython.display package that will allow us to see the images here (THIS IS NOT DONE IN ANALYSES):

In [ ]:
from IPython.display import Image

Image(filename="MyFirstPlot_SaveAs.png")

In [ ]:
Image(filename="MyFirstPlot_Print.png")

The following lines contain some parts of the code that can be used to better visualize the results:

In [ ]:
# Apply a selection (output will be the number of entries in the ):
ttree.Draw("Muon_pt", "Muon_pt > 10.0")

# Draw two things together:
ttree.Draw("Muon_pt", "Muon_pt > 50.0", "SAME")

# 2D drawing:
ttree.Draw("Muon_pt:Muon_eta")

# Only the first muon?
ttree.Draw("Muon_pt[0]")

# Or the second?
ttree.Draw("Muon_pt[1]")

# Set logarithmic scale in the canvas
canvas.SetLogy(1)

# To do:

Just to practice a little bit, there are some things that could be tried to get practice:

- For now we have only looked at the pt, but it would be nice to explore other basic quantities e.g. the azimuthal angle $\phi$, pseudorapidity $\eta$, charge, transverse impact parameter $d_{xy}$, isolation, number of hits... **Look at the TTree, find these variables and have a look, then answer the following questions:**
    - Why do you think $\eta$ has this shape?
    - Is this the behavior you expect from $\phi$? If yes/no, why?
    - How does the charge look like? Does it make sense to you? Why?
    - Look for the definition of the transverse impact parameter and also look at the distribution, does it make sense? What can be the muons with high $d_{xy}$?
    - Look at the charge of all the muons, why is it like this?
- Are the muons stored in an specific order? Why do you think is that? Does it happen the same for other variables?
- Another interesting quantities that are often studied when looking at dimuons are:
    - The relative directions between the muons ($\Delta\phi$, $\Delta\eta$, $\Delta R$, 3D angle...)
    - How is the charge of the muons? Why?
    - Which other variables do you find interesting?